**`Installing required libraries`**

In [1]:
%%capture

!pip install python-google-places
!pip install langdetect
!pip install bnlp_toolkit
!wget https://www.omicronlab.com/download/fonts/kalpurush.ttf
!wget https://www.omicronlab.com/download/fonts/Siyamrupali.ttf
!pip install folium
!pip install geopandas

**`Importing required Libraries`**

In [1]:
import re
import time
import html
import folium
import unicodedata
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from googleplaces import GooglePlaces, types, lang
from IPython.display import Markdown, display
from langdetect import detect
from folium.plugins import MarkerCluster  #marketcluster plugin
from folium.plugins import MousePosition  # MousePosition plugin
from folium.features import DivIcon  #DivIcon plugin   

**`Read the dataset`**

https://www.kaggle.com/mushfiqurrobin/network-coverage

In [2]:
df = pd.read_csv(r'Coverage.csv')
df.head()

,Index,Upazila_or_Thana,District,Area_Zip_Code,Latitude,Longitude,Operator,Active_Network_Available,Network_Level,Satisfaction_Score,Real_Time_Experience
0,0,Kawkhali,Pirojpur,8510,22.626128,90.059277,GP,4G,Low,3,4.0
1,1,Mathbaria,Pirojpur,8560,22.287592,89.959272,GP,4G,Low,3,4.0
2,2,Nazirpur,Pirojpur,8540,22.729367,89.960880,GP,4G,Low,3,5.0
3,3,Nesarabad,Pirojpur,8522,22.747200,90.103600,GP,4G,Moderate,6,6.0
4,4,Pirojpur Sadar,Pirojpur,8500,22.577800,89.990000,GP,4G,Good,10,7.5


In [3]:
area = df[['District', 'Upazila_or_Thana']]
area.head()

,District,Upazila_or_Thana
0,Pirojpur,Kawkhali
1,Pirojpur,Mathbaria
2,Pirojpur,Nazirpur
3,Pirojpur,Nesarabad
4,Pirojpur,Pirojpur Sadar


In [4]:
# Checking the Missing Values in total and percentage calculation

total = area.isnull().sum().sort_values(ascending=False)
percent = (area.isnull().sum()/area.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent*100], axis=1, keys=['total', 'percent'])
display(missing_data.head(5))

,total,percent
District,0,0.0
Upazila_or_Thana,0,0.0


In [5]:
# Checking Duplicate Rows 
area.duplicated().any()

True

In [6]:
area.duplicated().sum()

21800

In [7]:
## as there are duplicate values available, will be removing the duplicates

area.drop_duplicates(keep="first", inplace=True)
area.reset_index(drop=True, inplace=True)

area.to_csv("LocationsData.csv", index=False)

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [8]:
API_key = "----------------your API key ----------------------------"
google_places = GooglePlaces(API_key)


In [9]:
scraped_restaurants = []
radius = 0.5

# Converting the list of Upazilla/Thana and District into a combined string
locations = []
areaList = area.values.tolist()

for area in areaList:
    locationNames = ', '.join([str(item) for item in area])
    locations.append(locationNames)

print(locations)

['Pirojpur, Kawkhali', 'Pirojpur, Mathbaria', 'Pirojpur, Nazirpur', 'Pirojpur, Nesarabad', 'Pirojpur, Pirojpur Sadar', 'Pirojpur, Zianagar', 'Brahmanbaria, Akhaura', 'Brahmanbaria, Ashuganj', 'Brahmanbaria, Brahmanbaria Sadar', 'Brahmanbaria, Bancharampur', 'Brahmanbaria, Bijoynagar', 'Brahmanbaria, Kasba', 'Brahmanbaria, Nabinagar', 'Brahmanbaria, Nasirnagar', 'Brahmanbaria, Sarail', 'Bandarban, Alikadam', 'Bandarban, Bandarban Sadar', 'Bandarban, Lama', 'Bandarban, Naikhongchhari', 'Bandarban, Rowangchari', 'Bandarban, Ruma', 'Bandarban, Thanchi', 'Chandpur, Chandpur Sadar', 'Chandpur, Faridganj', 'Chandpur, Haimchar', 'Chandpur, Hajiganj', 'Chandpur, Kachua', 'Chandpur, Matlab (Dakshin)', 'Chandpur, Matlab (Uttar)', 'Chandpur, Shahrasti', 'Chittagong, Anwara', 'Chittagong, Banskhali', 'Chittagong, Boalkhali', 'Chittagong, Chandanish', 'Chittagong, Fatikchari', 'Chittagong, Karnaphuli', 'Chittagong, Lohagara', 'Chittagong, Mirsharai', 'Chittagong, Patiya', 'Chittagong, Rangunia', 'Ch

In [16]:
for location in locations:
    print(".............", location, "...............")
    location_result = google_places.nearby_search(location=location, keyword='Restaurant', radius=radius) 
    if location_result:
        for place in location_result.places:
            place.get_details()
            place_id = place.details.get('place_id')
            name = place.name
            latitude = place.geo_location.get('lat')
            longitude = place.geo_location.get('lng')
            rating = place.rating
            number_of_reviews = place.details.get('user_ratings_total')
            affluence = place.details.get('price_level')
            address = place.formatted_address

            scraped_restaurants.append([place_id, name, latitude, longitude, rating, number_of_reviews, affluence, address])
            # print(place.details)

            # print(restaurant_data)
            print(".................. Scrapped Restaurants: ", len(scraped_restaurants))
            time.sleep(5) 

            while location_result.has_next_page_token:
                query_result = google_places.nearby_search(location=location, keyword='Restaurant',
                radius=radius, pagetoken=location_result.next_page_token)
                
                for place in location_result.places:
                    place.get_details()
                    place_id = place.details.get('place_id')
                    name = place.name
                    latitude = place.geo_location.get('lat')
                    longitude = place.geo_location.get('lng')
                    rating = place.rating
                    number_of_reviews = place.details.get('user_ratings_total')
                    affluence = place.details.get('price_level')
                    address = place.formatted_address

                    scraped_restaurants.append([place_id, name, latitude, longitude, rating, number_of_reviews, affluence, address])
                     # print(place.details)
                    # print(restaurant_data)  
                    print(".................. Scrapped Restaurants: ", len(scraped_restaurants))
                    time.sleep(5) 
                    

            time.sleep(5)

# Dumping the data into a DataFrame
df_restaurant = pd.DataFrame(restaurant_data, columns=['place_id', 'name', 'latitude', 'longitude', 'rating', 'number_of_reviews', 'affluence', 'address'])

df_restaurant.to_csv("restaurants.csv", index=False, encoding='utf-8')

............. Pirojpur, Kawkhali ...............
............. Pirojpur, Mathbaria ...............
.................. Scrapped Restaurants:  1
.................. Scrapped Restaurants:  2
.................. Scrapped Restaurants:  3
.................. Scrapped Restaurants:  4
............. Pirojpur, Nazirpur ...............
.................. Scrapped Restaurants:  5
............. Pirojpur, Nesarabad ...............
.................. Scrapped Restaurants:  6
.................. Scrapped Restaurants:  7
.................. Scrapped Restaurants:  8
............. Pirojpur, Pirojpur Sadar ...............
.................. Scrapped Restaurants:  9
.................. Scrapped Restaurants:  10
.................. Scrapped Restaurants:  11
............. Pirojpur, Zianagar ...............
............. Brahmanbaria, Akhaura ...............
.................. Scrapped Restaurants:  12
.................. Scrapped Restaurants:  13
.................. Scrapped Restaurants:  14
.................. Scrapp

.................. Scrapped Restaurants:  162
.................. Scrapped Restaurants:  163
.................. Scrapped Restaurants:  164
.................. Scrapped Restaurants:  165
.................. Scrapped Restaurants:  166
.................. Scrapped Restaurants:  167
.................. Scrapped Restaurants:  168
.................. Scrapped Restaurants:  169
.................. Scrapped Restaurants:  170
.................. Scrapped Restaurants:  171
.................. Scrapped Restaurants:  172
.................. Scrapped Restaurants:  173
.................. Scrapped Restaurants:  174
.................. Scrapped Restaurants:  175
.................. Scrapped Restaurants:  176
.................. Scrapped Restaurants:  177
.................. Scrapped Restaurants:  178
.................. Scrapped Restaurants:  179
.................. Scrapped Restaurants:  180
.................. Scrapped Restaurants:  181
.................. Scrapped Restaurants:  182
.................. Scrapped Restau

.................. Scrapped Restaurants:  341
.................. Scrapped Restaurants:  342
.................. Scrapped Restaurants:  343
.................. Scrapped Restaurants:  344
.................. Scrapped Restaurants:  345
.................. Scrapped Restaurants:  346
.................. Scrapped Restaurants:  347
.................. Scrapped Restaurants:  348
.................. Scrapped Restaurants:  349
.................. Scrapped Restaurants:  350
.................. Scrapped Restaurants:  351
.................. Scrapped Restaurants:  352
.................. Scrapped Restaurants:  353
.................. Scrapped Restaurants:  354
.................. Scrapped Restaurants:  355
.................. Scrapped Restaurants:  356
.................. Scrapped Restaurants:  357
.................. Scrapped Restaurants:  358
.................. Scrapped Restaurants:  359
.................. Scrapped Restaurants:  360
.................. Scrapped Restaurants:  361
.................. Scrapped Restau

.................. Scrapped Restaurants:  520
.................. Scrapped Restaurants:  521
.................. Scrapped Restaurants:  522
.................. Scrapped Restaurants:  523
.................. Scrapped Restaurants:  524
.................. Scrapped Restaurants:  525
.................. Scrapped Restaurants:  526
.................. Scrapped Restaurants:  527
.................. Scrapped Restaurants:  528
.................. Scrapped Restaurants:  529
.................. Scrapped Restaurants:  530
.................. Scrapped Restaurants:  531
.................. Scrapped Restaurants:  532
.................. Scrapped Restaurants:  533
.................. Scrapped Restaurants:  534
.................. Scrapped Restaurants:  535
.................. Scrapped Restaurants:  536
.................. Scrapped Restaurants:  537
.................. Scrapped Restaurants:  538
.................. Scrapped Restaurants:  539
.................. Scrapped Restaurants:  540
.................. Scrapped Restau

.................. Scrapped Restaurants:  699
.................. Scrapped Restaurants:  700
.................. Scrapped Restaurants:  701
.................. Scrapped Restaurants:  702
.................. Scrapped Restaurants:  703
.................. Scrapped Restaurants:  704
.................. Scrapped Restaurants:  705
.................. Scrapped Restaurants:  706
.................. Scrapped Restaurants:  707
.................. Scrapped Restaurants:  708
.................. Scrapped Restaurants:  709
.................. Scrapped Restaurants:  710
.................. Scrapped Restaurants:  711
.................. Scrapped Restaurants:  712
.................. Scrapped Restaurants:  713
.................. Scrapped Restaurants:  714
.................. Scrapped Restaurants:  715
.................. Scrapped Restaurants:  716
.................. Scrapped Restaurants:  717
.................. Scrapped Restaurants:  718
.................. Scrapped Restaurants:  719
.................. Scrapped Restau

...

.................. Scrapped Restaurants:  1580
.................. Scrapped Restaurants:  1581
.................. Scrapped Restaurants:  1582
.................. Scrapped Restaurants:  1583
.................. Scrapped Restaurants:  1584
.................. Scrapped Restaurants:  1585
.................. Scrapped Restaurants:  1586
.................. Scrapped Restaurants:  1587
.................. Scrapped Restaurants:  1588
.................. Scrapped Restaurants:  1589
.................. Scrapped Restaurants:  1590
.................. Scrapped Restaurants:  1591
.................. Scrapped Restaurants:  1592
.................. Scrapped Restaurants:  1593
.................. Scrapped Restaurants:  1594
.................. Scrapped Restaurants:  1595
.................. Scrapped Restaurants:  1596
.................. Scrapped Restaurants:  1597
.................. Scrapped Restaurants:  1598
.................. Scrapped Restaurants:  1599
.................. Scrapped Restaurants:  1600
.............

.................. Scrapped Restaurants:  1755
.................. Scrapped Restaurants:  1756
.................. Scrapped Restaurants:  1757
.................. Scrapped Restaurants:  1758
.................. Scrapped Restaurants:  1759
.................. Scrapped Restaurants:  1760
.................. Scrapped Restaurants:  1761
.................. Scrapped Restaurants:  1762
.................. Scrapped Restaurants:  1763
.................. Scrapped Restaurants:  1764
.................. Scrapped Restaurants:  1765
.................. Scrapped Restaurants:  1766
.................. Scrapped Restaurants:  1767
.................. Scrapped Restaurants:  1768
.................. Scrapped Restaurants:  1769
.................. Scrapped Restaurants:  1770
.................. Scrapped Restaurants:  1771
.................. Scrapped Restaurants:  1772
.................. Scrapped Restaurants:  1773
.................. Scrapped Restaurants:  1774
.................. Scrapped Restaurants:  1775
.............

.................. Scrapped Restaurants:  1930
.................. Scrapped Restaurants:  1931
.................. Scrapped Restaurants:  1932
.................. Scrapped Restaurants:  1933
.................. Scrapped Restaurants:  1934
.................. Scrapped Restaurants:  1935
.................. Scrapped Restaurants:  1936
.................. Scrapped Restaurants:  1937
.................. Scrapped Restaurants:  1938
.................. Scrapped Restaurants:  1939
.................. Scrapped Restaurants:  1940
.................. Scrapped Restaurants:  1941
.................. Scrapped Restaurants:  1942
.................. Scrapped Restaurants:  1943
.................. Scrapped Restaurants:  1944
.................. Scrapped Restaurants:  1945
.................. Scrapped Restaurants:  1946
.................. Scrapped Restaurants:  1947
.................. Scrapped Restaurants:  1948
.................. Scrapped Restaurants:  1949
.................. Scrapped Restaurants:  1950
.............

.................. Scrapped Restaurants:  2105
.................. Scrapped Restaurants:  2106
.................. Scrapped Restaurants:  2107
.................. Scrapped Restaurants:  2108
.................. Scrapped Restaurants:  2109
.................. Scrapped Restaurants:  2110
.................. Scrapped Restaurants:  2111
.................. Scrapped Restaurants:  2112
.................. Scrapped Restaurants:  2113
.................. Scrapped Restaurants:  2114
.................. Scrapped Restaurants:  2115
.................. Scrapped Restaurants:  2116
.................. Scrapped Restaurants:  2117
.................. Scrapped Restaurants:  2118
.................. Scrapped Restaurants:  2119
.................. Scrapped Restaurants:  2120
.................. Scrapped Restaurants:  2121
.................. Scrapped Restaurants:  2122
.................. Scrapped Restaurants:  2123
.................. Scrapped Restaurants:  2124
.................. Scrapped Restaurants:  2125
.............

KeyboardInterrupt: 

read more

**Documentations :**
- https://medium.com/swlh/scraping-google-maps-using-selenium-3cec08eb6a92
- https://towardsdatascience.com/creating-a-dataset-using-an-api-with-python-dcc1607616d
- https://outscraper.com/scrape-google-maps-in-python/
- https://towardsdatascience.com/foods-around-me-google-maps-data-scraping-with-python-google-colab-588986c63db3
- https://app.outscraper.com/api-docs
- Scrape Google Maps Data Legally via Official API : https://stevesie.com/apps/google-maps-api

- https://scrapingrobot.com/blog/google-places-scraper/
- https://www.octoparse.com/blog/google-maps-crawlers
- https://medium.com/@cam.bharath92/web-scrape-google-places-using-google-place-api-af003a4fb00e
- https://scrape.do/blog
